In [ ]:
! pip install -q unsloth
! pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
! pip install -q datasets accelerate bitsandbytes trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 745.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Load model with Unsloth (4-bit QLoRA)

In [ ]:
! pip install unsloth_zoo

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Maximum context length (tokens) the model will accept
dtype = None           # auto-detect (torch.float16 or torch.bfloat16)
load_in_4bit = True    # QLoRA

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.2-3b-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

### Apply LoRA adapters (VERY IMPORTANT)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)


Unsloth: Already have LoRA adapters! We shall skip this step.


### Load your dataset (JSONL)

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/training_data_fixed.jsonl",
    split="train"
)

print(dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

{'messages': [{'role': 'user', 'content': "What does 'Return' mean in the context of a library's cataloging system?"}, {'role': 'assistant', 'content': 'The act of sending an item back to its owner or the library, often due to being lost, damaged, or no longer needed.'}]}


Convert to training text (Llama-3 chat format)

In [ ]:
def format_chat(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False
        )
    }

dataset = dataset.map(format_chat)


Map:   0%|          | 0/255 [00:00<?, ? examples/s]

### Finding efficient Tokens

In [ ]:
def count_tokens(dataset, tokenizer, field="text"):
    return sum(len(tokenizer(example[field]).input_ids)
               for example in dataset)


In [ ]:
print(count_tokens(dataset, tokenizer))

20697


### Fine-tune using TRL SFTTrainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100,              # 🔹 increase later
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/255 [00:00<?, ? examples/s]

### Start training

In [ ]:
trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 255 | Num Epochs = 4 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
10,3.608400
20,1.823000
30,1.485200
40,1.295000
50,1.176600
60,1.047600
70,0.937300
80,0.917000
90,1.004500
100,0.845000


TrainOutput(global_step=100, training_loss=1.4139514636993409, metrics={'train_runtime': 218.6381, 'train_samples_per_second': 3.659, 'train_steps_per_second': 0.457, 'total_flos': 1232739984887808.0, 'train_loss': 1.4139514636993409, 'epoch': 3.125})

### Save LoRA adapter

In [ ]:
model.save_pretrained("llama3-qlora-adapter")
tokenizer.save_pretrained("llama3-qlora-adapter")


('llama3-qlora-adapter/tokenizer_config.json',
 'llama3-qlora-adapter/special_tokens_map.json',
 'llama3-qlora-adapter/chat_template.jinja',
 'llama3-qlora-adapter/tokenizer.json')

### Inference test

In [ ]:
FastLanguageModel.for_inference(model)

prompt = "How many entities are there in this project?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 400,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


system

Cutting Knowledge Date: December 2023
Today Date: 10 Jan 2026

user

How many entities are there in this project?assistant

There are 10 entities in this project: Users, Roles, Permissions, Categories, Products, Orders, Seats, SeatTypes, SeatStatuses, and Payments.


In [ ]:
FastLanguageModel.for_inference(model)

prompt = "What is the role of User class?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 400,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


system

Cutting Knowledge Date: December 2023
Today Date: 10 Jan 2026

user

What is the role of User class?assistant

The User class represents a user entity and is used to validate and map user input.


In [ ]:
FastLanguageModel.for_inference(model)

prompt = "Explain how spring security implement in this project in detail?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 2000,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


system

Cutting Knowledge Date: December 2023
Today Date: 10 Jan 2026

user

Explain how spring security implement in this project in detail?assistant

Spring Security is implemented using JavaConfig and @EnableWebSecurity annotations, which configure the security settings for the application. It includes authentication, authorization, and access control. The Spring Security framework provides a lot of built-in functionality for handling these tasks, making it easier to secure an application.


In [ ]:
FastLanguageModel.for_inference(model)

prompt = "Explain the workflow of this project?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 2000,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


system

Cutting Knowledge Date: December 2023
Today Date: 10 Jan 2026

user

Explain the workflow of this project?assistant

The workflow involves a user sending a booking request, which is received by an iLibrary-Backend service. This service then calls a BookingController, which queries the SeatService to find available seats. If available, the booking is created and the SeatService updates the seat status.
